# Object recognition using CIFAR10 dataset

In this project, we identify objects from collection of 60,000 color images of 10 object classes.
- Data source: http://www.cs.toronto.edu/~kriz/cifar.html
- The images are color graphics
- Each image is of 32 x 32 pixel size
- Data is about 163MB
- We initially apply a simple neural network model to baseline
- Apply convolution, pooling and dropout to see performance improvement
- Increase the number of layers to see if performance improves

We apply

- convolution to capture patterns
- pooling to reduce the size of image
- dropout to generalize model

In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt

In [2]:
(X_train, y_train), (X_valid, y_valid) = cifar10.load_data()

X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')

# normalize inputs ranging between (0 - 255) to between (0 - 1)
X_train = X_train / 255.0
X_valid = X_valid / 255.0

# one hot encode outputs
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

num_classes = y_valid.shape[1]

170500096/170498071 [==============================] - 4s 0us/step


__Simple CNN__

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu', kernel_constraint=max_norm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_constraint=max_norm(3)))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=max_norm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

epochs = 25
lrate = 0.01
decay = lrate / epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)              

In [8]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=32)
scores = model.evaluate(X_valid, y_valid, verbose=0)

print('Accuracy = %.2f%%' % (scores[1] * 100))

Epoch 1/25
1563/1563 [==============================] - 15s 5ms/step - loss: 1.9146 - accuracy: 0.3023 - val_loss: 1.4192 - val_accuracy: 0.4876
Epoch 2/25
1563/1563 [==============================] - 8s 5ms/step - loss: 1.3952 - accuracy: 0.4982 - val_loss: 1.2700 - val_accuracy: 0.5422
Epoch 3/25
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2119 - accuracy: 0.5702 - val_loss: 1.1255 - val_accuracy: 0.6047
Epoch 4/25
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0787 - accuracy: 0.6165 - val_loss: 1.0463 - val_accuracy: 0.6260
Epoch 5/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9825 - accuracy: 0.6482 - val_loss: 1.0013 - val_accuracy: 0.6421
Epoch 6/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8985 - accuracy: 0.6831 - val_loss: 0.9578 - val_accuracy: 0.6633
Epoch 7/25
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8126 - accuracy: 0.7123 - val_loss: 0.9374 - val_accuracy

__Larger CNN__

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=max_norm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=max_norm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

epochs = 25
lrate = 0.01
decay = lrate / epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_8 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)       

In [12]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=64)
scores = model.evaluate(X_valid, y_valid, verbose=0)

print('Accuracy = %.2f%%' % (scores[1] * 100))

Epoch 1/25
782/782 [==============================] - 8s 10ms/step - loss: 2.1145 - accuracy: 0.2062 - val_loss: 1.5944 - val_accuracy: 0.4190
Epoch 2/25
782/782 [==============================] - 7s 9ms/step - loss: 1.5693 - accuracy: 0.4278 - val_loss: 1.3697 - val_accuracy: 0.5086
Epoch 3/25
782/782 [==============================] - 7s 9ms/step - loss: 1.3665 - accuracy: 0.5035 - val_loss: 1.2297 - val_accuracy: 0.5610
Epoch 4/25
782/782 [==============================] - 7s 9ms/step - loss: 1.2308 - accuracy: 0.5567 - val_loss: 1.1679 - val_accuracy: 0.5801
Epoch 5/25
782/782 [==============================] - 7s 9ms/step - loss: 1.1174 - accuracy: 0.5969 - val_loss: 1.0297 - val_accuracy: 0.6336
Epoch 6/25
782/782 [==============================] - 7s 9ms/step - loss: 1.0160 - accuracy: 0.6371 - val_loss: 0.9627 - val_accuracy: 0.6583
Epoch 7/25
782/782 [==============================] - 7s 9ms/step - loss: 0.9372 - accuracy: 0.6652 - val_loss: 0.8968 - val_accuracy: 0.6867
Epoch